In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

Reading the Data

In [ ]:
data = "C:/Users/yasht/OneDrive/Desktop/Stratascratch/Insights from City Supply and Demand Data/datasets/Insights from City Supply and Demand Data.csv"
df = pd.read_csv(data)
df.head()

Question 1


Which date had the most completed trips during the two week period?



In [ ]:
df.fillna(method='ffill')

In [ ]:
df_agg_date = df.groupby('Date').sum().reset_index()
df_agg_date

In [ ]:
df_agg_date.nlargest(1,'Completed Trips ')['Date']

Question 2


What was the highest number of completed trips within a 24 hour period?


In [ ]:
df_1 =df.fillna(method='ffill')

In [ ]:
def create_timestamp(date, time):
    return pd.to_datetime(f"{date} {time}:00")
df_1['Timestamp'] = df_1.apply(lambda row: create_timestamp(row['Date'], row['Time (Local)']), axis=1)

In [ ]:
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=24)
df_agg_twentyfour_hrs = df_1.set_index("Timestamp").rolling(window=indexer, min_periods=5).sum()
df_agg_twentyfour_hr = df_1.set_index("Timestamp").rolling(window=indexer, min_periods=1).sum()


In [ ]:
df_agg_twentyfour_hr[:10]
time_format = "%Y/%m/%d %H:%M"
df_agg_twentyfour_hrs['Time Interval']  = df_agg_twentyfour_hrs.index.strftime(time_format) +  " - " + (df_agg_twentyfour_hrs.index + pd.Timedelta('24 hours')).strftime(time_format)

In [ ]:
completed_trips = df_agg_twentyfour_hrs.nlargest(1, 'Completed Trips ')['Completed Trips '].values[0]
completed_trips


In [ ]:
time_interval = df_agg_twentyfour_hrs.nlargest(1, 'Completed Trips ')['Time Interval'].values[0]
time_interval

In [ ]:
print("Number of completed trips:", completed_trips)
print("Time Interval:", time_interval)

Question 3 

Which hour of the day had the most requests during the two week period?



In [219]:
df_agg_time = df.groupby('Time (Local)').sum().reset_index()
df_agg_time.nlargest(1, 'Requests ')

df_agg_time.nlargest(1, 'Requests ')['Time (Local)']


23    23
Name: Time (Local), dtype: int64

Question 4


What percentages of all zeroes during the two week period occurred on weekend (Friday at 5 pm to Sunday at 3 am)?



In [ ]:
df_4 = df.fillna(method='ffill')

In [ ]:
total_zeros = df_4['Zeroes '].sum()
total_zeros

In [ ]:
df_4['Day'] = pd.to_datetime(df_4['Date']).dt.weekday

In [ ]:

weekend_zeros = df_4[((df_4['Day'] == 4) & (df_4['Time (Local)'] >= 17)) | (df_4['Day'] == 5) | (((df_4['Day'] == 6) & (df_4['Time (Local)'] < 3)))]['Zeroes '].sum()
weekend_zeros

In [ ]:
weekend_zeros_percent = weekend_zeros / total_zeros * 100
print(weekend_zeros_percent,'%')

Question 5


What is the weighted average ratio of completed trips per driver during the two week period?

In [ ]:
df_5 = df.fillna(method='ffill')

In [ ]:
df_5['completed_trip_ratio_per_driver'] =  df_5['Completed Trips '] / df_5['Unique Drivers']

In [ ]:
data_without_zero_unique_driver = df_5[df_5['Unique Drivers'] > 0]

In [ ]:
weighted_average_ratio = np.average(data_without_zero_unique_driver['completed_trip_ratio_per_driver'], weights= data_without_zero_unique_driver['Completed Trips '])
weighted_average_ratio

In [ ]:
np.average(data_without_zero_unique_driver['completed_trip_ratio_per_driver'])


Question 6 



In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours over the two week period in terms of unique requests? A new shift starts in every 8 hours. Assume that a driver will work same shift each day.


In [ ]:
df_6 = df.fillna(method='ffill')

In [ ]:
df_6['Timestamp'] = df_6.apply(lambda row: create_timestamp(row['Date'], row['Time (Local)']), axis=1)


In [ ]:
df_6 = df_6.set_index("Timestamp").resample('8H').sum()

In [ ]:
df_6['Shift'] = df_6.index.strftime(time_format) + ' - ' + (df_6.index + pd.Timedelta('8 Hours')).strftime(time_format)



In [220]:
df_6.nlargest(1, 'Requests ')['Shift']



Timestamp
2012-09-21 16:00:00    2012/09/21 16:00 - 2012/09/22 00:00
Freq: 8H, Name: Shift, dtype: object

Question 7


True or False: Driver supply always increases when demand increases during the two week period

In [ ]:
df_7 = df.fillna(method='ffill')

In [ ]:
df_7['Timestamp'] = df_7.apply(lambda row: create_timestamp(row['Date'], row['Time (Local)']), axis=1)

In [ ]:
diff_df = pd.DataFrame(df_7['Timestamp'])

In [ ]:
diff_df['request diff'] = df_7['Requests '].shift(-1) - df_7['Requests ']

In [ ]:
diff_df['supply diff'] = df_7['Unique Drivers'].shift(-1) - df_7['Unique Drivers']

In [ ]:
(diff_df[diff_df['request diff'] > 0]['supply diff'] > 0).all().item()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,8))
sns.lineplot(data=df_7[['Requests ', 'Unique Drivers', 'Timestamp']])
plt.show()

In [ ]:
df_8

Question 8


In which 72 hour period is the ratio of Zeroes to Eyeballs the highest?



In [ ]:
df_8 = df.fillna(method= 'ffill')

In [ ]:
df_8['Timestamp'] = df_8.apply(lambda row: create_timestamp(row['Date'], row['Time (Local)']), axis = 1)


In [ ]:
df_8

In [ ]:
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=72)

df_agg_three_day = df_8.set_index('Timestamp').rolling(window=indexer, min_periods = 1).sum().reset_index()

In [ ]:
df_agg_three_day['zero_to_eyeballs_ratio'] = df_agg_three_day['Zeroes '] / df_agg_three_day['Eyeballs ']

In [ ]:
df_agg_three_day = df_agg_three_day[df_agg_three_day['Timestamp'] <= (df_agg_three_day['Timestamp'].max() - pd.Timedelta(days=3))]


In [ ]:
df_agg_three_day[:15]

In [ ]:
df_agg_three_day['Time Interval'] = df_agg_three_day['Timestamp'].dt.strftime(time_format) + ' - ' + (df_agg_three_day['Timestamp'] + pd.Timedelta('3 days')).dt.strftime(time_format)

In [ ]:
df_agg_three_day.nlargest(1, 'zero_to_eyeballs_ratio')['Time Interval']

In [ ]:
df_agg_three_day.nlargest(1, 'zero_to_eyeballs_ratio')['zero_to_eyeballs_ratio']


In [ ]:
df_agg_three_day['zero_to_eyeballs_ratio'].max()

Question 9


If you could add 5 drivers to any single hour of every day during the two week period, which hour should you add them to? Hint: Consider both rider eyeballs and driver supply when choosing


In [ ]:
df_9 = df.fillna(method='ffill')

In [ ]:
df_9 = df_9.groupby('Time (Local)').sum().reset_index()

In [ ]:
df_9['eye_to_driver_ratio'] = df_9['Eyeballs '] / df_9['Unique Drivers']
df_9

In [ ]:
df_9.nlargest(1, 'eye_to_driver_ratio')['Time (Local)']

Question 10


True or False: There is exactly two weeks of data in this analysis


In [ ]:
df_8['Timestamp'].max() - df_8['Timestamp'].min()
pd.Timedelta('14 Days') == df_8['Timestamp'][df_8.shape[0]-1] - df_8['Timestamp'][0]
pd.Timedelta('14 Days') > df_8['Timestamp'][df_8.shape[0]-1] - df_8['Timestamp'][0]

Question 11


Looking at the data from all two weeks, which time might make the most sense to consider a true "end day" instead of midnight? (i.e when are supply and demand at both their natural minimums) Tip: Visualize the data to confirm your answer if needed.


In [ ]:
df_8 = df.fillna(method='ffill')
df_0 = df_8.groupby(by = 'Time (Local)').sum()
min_point = df_0['Requests '].min()
min_point_value = df_0['Unique Drivers'].min()

In [ ]:
plt.figure(figsize=(12,8))

fig = sns.lineplot(data=df_0[['Requests ', 'Unique Drivers']], markers=True, dashes=False)
fig.set_xticks(df_0.index)

plt.annotate('minimum supply/demand', xy=(min_point, min_point_value), color='r', fontsize=16)
